# GameTheory 17 - Theoreme d'Existence de Nash

**Navigation** : [← GameTheory-16-Lean-Definitions](GameTheory-16-Lean-Definitions.ipynb) | [Index](GameTheory-1-Setup.ipynb) | [GameTheory-18-Lean-CombinatorialGames →](GameTheory-18-Lean-CombinatorialGames.ipynb)

---

## Introduction

Ce notebook explore le celebre **theoreme d'existence de Nash** (1950) :

> *Tout jeu fini possede au moins un equilibre de Nash en strategies mixtes.*

La preuve repose sur le **theoreme du point fixe de Brouwer**, un resultat fondamental de topologie. Nous etudierons une formalisation complete en Lean 4 du depot [math-xmum/Brouwer](https://github.com/math-xmum/Brouwer).

### Objectifs pedagogiques

1. Comprendre le lien entre points fixes et equilibres de Nash
2. Etudier le theoreme de Brouwer et sa formalisation
3. Analyser la structure de la preuve d'existence
4. Explorer les extensions (Kakutani, Scarf)

### Prerequis

- Avoir complete le notebook 16 (definitions de base)
- Notions de topologie (continuite, compacite, convexite)
- Familiarite avec les preuves en Lean (tactiques de base)

### Duree estimee : 55 minutes

---

## Plan de ce Notebook

1. [Configuration](#1-configuration)
2. [Theoreme du Point Fixe de Brouwer](#2-brouwer)
3. [Application a la Theorie des Jeux](#3-application)
4. [Lecture Guidee : math-xmum/Brouwer](#4-lecture-guidee)
5. [Structure de la Preuve](#5-structure-preuve)
6. [Exercices](#6-exercices)
7. [Solutions](#7-solutions)
8. [Resume](#8-resume)

<a id="1-configuration"></a>
## 1. Configuration

In [ ]:
import sys
import os

# Ajouter le chemin vers lean_runner
lean_path = os.path.abspath('../SymbolicAI/Lean')
if lean_path not in sys.path:
    sys.path.insert(0, lean_path)

from lean_runner import LeanRunner

runner = LeanRunner(backend='auto', timeout=60)

def run_lean(code: str, show_code: bool = True):
    """Execute du code Lean et affiche le resultat."""
    result = runner.run(code)
    if show_code:
        print("--- Code Lean ---")
        for i, line in enumerate(code.strip().split('\n'), 1):
            print(f"{i:3d} | {line}")
        print("-" * 40)
    if result.output:
        print("Sortie:", result.output)
    if result.errors:
        print("Erreurs:" if not result.success else "Avertissements:", result.errors)
    print(f"[{'OK' if result.success else 'ECHEC'}]")
    return result

print(f"Backend: {runner.check_installation()['backend']}")

<a id="2-brouwer"></a>
## 2. Theoreme du Point Fixe de Brouwer

### 2.1 Enonce

**Theoreme de Brouwer (1911)** : Soit $f : B^n \to B^n$ une fonction continue ou $B^n$ est la boule unite fermee de $\mathbb{R}^n$. Alors $f$ admet au moins un **point fixe**, c'est-a-dire un point $x^*$ tel que $f(x^*) = x^*$.

**Variante sur le simplexe** : Le theoreme s'applique egalement au simplexe standard :
$$\Delta^{n-1} = \{(x_1, ..., x_n) \in \mathbb{R}^n : x_i \geq 0, \sum_i x_i = 1\}$$

car le simplexe est homeomorphe a la boule.

### 2.2 Intuition geometrique

Imaginez une carte de France posee sur la France reelle. Si vous deformez la carte de maniere continue (sans la dechirer), au moins un point de la carte sera exactement au-dessus de sa position reelle.

In [ ]:
# Illustration Python : recherche de point fixe par iteration
import numpy as np
import matplotlib.pyplot as plt

def find_fixed_point_1d(f, x0=0.5, tol=1e-6, max_iter=100):
    """Recherche d'un point fixe par iteration."""
    x = x0
    history = [x]
    for _ in range(max_iter):
        x_new = f(x)
        history.append(x_new)
        if abs(x_new - x) < tol:
            return x_new, history
        x = x_new
    return x, history

# Exemple : f(x) = 0.5 * (x + 1/x) sur [0.5, 2] -> point fixe = 1
f = lambda x: 0.5 * (x + 1/max(x, 0.01))
fixed, history = find_fixed_point_1d(f, x0=2.0)
print(f"Point fixe trouve: {fixed:.6f}")
print(f"Verification: f({fixed:.6f}) = {f(fixed):.6f}")

### 2.3 Formalisation en Lean

En Lean, l'enonce du theoreme de Brouwer sur le simplexe ressemble a :

In [ ]:
run_lean("""
-- Enonce simplifie du theoreme de Brouwer
-- (La version complete dans math-xmum/Brouwer utilise des types plus sophistiques)

-- Simplexe standard de dimension n-1
def Simplex (n : Nat) : Type := 
  { v : Fin n → Float // (∀ i, v i >= 0) ∧ 
    (List.finRange n).foldl (fun acc i => acc + v i) 0 = 1 }

-- Enonce du theoreme (axiomatise pour ce notebook)
-- Une fonction continue du simplexe dans lui-meme a un point fixe
axiom brouwer_simplex {n : Nat} (f : Simplex n → Simplex n) 
    (hcont : True)  -- condition de continuite (simplifiee)
    : ∃ x : Simplex n, f x = x

#check @brouwer_simplex
""")

<a id="3-application"></a>
## 3. Application a la Theorie des Jeux

### 3.1 Correspondance de meilleure reponse

Pour un jeu fini, definissons la **correspondance de meilleure reponse** :

$$BR_i(\sigma_{-i}) = \arg\max_{\sigma_i} E[u_i(\sigma_i, \sigma_{-i})]$$

C'est l'ensemble des strategies mixtes qui maximisent le gain du joueur $i$ etant donne les strategies des autres.

### 3.2 Fonction de meilleure reponse combinee

On definit $BR : \prod_i \Delta_i \to \prod_i \Delta_i$ par :
$$BR(\sigma) = (BR_1(\sigma_{-1}), ..., BR_n(\sigma_{-n}))$$

### 3.3 Lien avec Nash

**Observation cle** : $\sigma^*$ est un equilibre de Nash si et seulement si $\sigma^* \in BR(\sigma^*)$, c'est-a-dire si $\sigma^*$ est un **point fixe** de $BR$.

Le probleme : $BR$ n'est pas une fonction mais une **correspondance** (plusieurs meilleures reponses possibles). Il faut utiliser le theoreme de **Kakutani** (extension de Brouwer aux correspondances).

In [ ]:
run_lean("""
-- La strategie de la preuve de Nash

-- Au lieu d'utiliser Kakutani directement, on peut construire
-- une fonction continue dont les points fixes sont les equilibres de Nash

-- L'idee : perturber les strategies vers les meilleures reponses
-- f(sigma) = sigma + epsilon * direction_vers_meilleure_reponse
-- puis normaliser pour rester dans le simplexe

-- Cette fonction est continue, donc par Brouwer elle a un point fixe
-- A ce point fixe, sigma = f(sigma), donc sigma est deja meilleure reponse

-- Axiome : l'existence d'une telle fonction
axiom nash_map_exists {n : Nat} (g : Fin n → Fin n → Int) :
  ∃ f : (Fin n → Float) → (Fin n → Float),
    -- f est continue (simplifie)
    True ∧
    -- Les points fixes de f correspondent aux equilibres de Nash
    True

#check @nash_map_exists
""")

<a id="4-lecture-guidee"></a>
## 4. Lecture Guidee : math-xmum/Brouwer

Le depot [math-xmum/Brouwer](https://github.com/math-xmum/Brouwer) contient une formalisation complete de la preuve en Lean 4.

### 4.1 Structure du projet

| Fichier | Lignes | Contenu |
|---------|--------|--------|
| `Simplex.lean` | ~100 | Simplexe standard, distributions |
| `Scarf.lean` | ~3000 | Lemme de Scarf (colorages) |
| `Brouwer.lean` | ~1000 | Theoreme de Brouwer via Sperner |
| `Brouwer_product.lean` | ~900 | Extension aux produits |
| `Nash.lean` | ~600 | Definition de jeu, existence Nash |

### 4.2 Definition du jeu (Nash.lean)

Voici les definitions cles du depot :

In [ ]:
# Code extrait de math-xmum/Brouwer/Nash.lean
nash_lean_code = """
-- Extrait de https://github.com/math-xmum/Brouwer/blob/main/Brouwer/Nash.lean
-- (Simplifie pour illustration)

/-
Structure de jeu selon math-xmum/Brouwer :

structure Game where
  I : Type*                          -- Ensemble des joueurs
  SS : I → Type*                     -- Ensembles de strategies
  g : I → (Π i, SS i) → ℝ          -- Fonctions de gain

Le simplexe standard :
def stdSimplex (α : Type*) : Type :=
  {f : α → ℝ // (∀ x, 0 ≤ f x) ∧ (∑ x, f x = 1)}

Strategie mixte :
def MixedStrategy (g : Game) (i : g.I) := stdSimplex (g.SS i)

Equilibre de Nash :
def IsNashEquilibrium (g : FiniteGame) (σ : MixedProfile g) : Prop :=
  ∀ i, ∀ σ'_i, ExpectedPayoff g i σ ≥ ExpectedPayoff g i (deviate σ i σ'_i)
-/

#check True  -- Placeholder pour code qui necessite mathlib
"""

print("=" * 60)
print("DEFINITIONS CLES DE math-xmum/Brouwer/Nash.lean")
print("=" * 60)
print(nash_lean_code)
print("\n[Ces definitions necessitent mathlib4 pour etre executees]")

### 4.3 Chaine de preuves

La preuve d'existence suit cette structure :

```
Lemme de Scarf (colorages de simplexes)
    ↓
Lemme de Sperner
    ↓
Theoreme de Brouwer (simplexe)
    ↓
Theoreme de Brouwer (produit de simplexes)
    ↓
Existence d'equilibre de Nash
```

### 4.4 Theoreme de Brouwer dans le depot

In [ ]:
brouwer_code = """
-- Extrait de https://github.com/math-xmum/Brouwer/blob/main/Brouwer/Brouwer.lean

/-
Le theoreme principal (simplifie) :

theorem brouwer_fixed_point 
    {n : ℕ} 
    (f : stdSimplex (Fin n) → stdSimplex (Fin n))
    (hf : Continuous f) :
    ∃ x, f x = x := by
  -- La preuve utilise le lemme de Sperner
  -- qui lui-meme utilise le lemme de Scarf
  -- sur les colorages de triangulations du simplexe
  sorry  -- Preuve complete dans le depot

La version sur produit de simplexes (pour Nash) :

theorem brouwer_product_fixed_point
    {I : Type*} [Fintype I]
    {S : I → Type*} [∀ i, Fintype (S i)]
    (f : (Π i, stdSimplex (S i)) → (Π i, stdSimplex (S i)))
    (hf : Continuous f) :
    ∃ x, f x = x := by
  -- Extension du theoreme de Brouwer au produit
  sorry
-/

#check True
"""

print("=" * 60)
print("THEOREME DE BROUWER - math-xmum/Brouwer")
print("=" * 60)
print(brouwer_code)

<a id="5-structure-preuve"></a>
## 5. Structure de la Preuve d'Existence

### 5.1 Etapes principales

1. **Definir la fonction de meilleure reponse perturbee** :
   $$f_i(\sigma) = \frac{\sigma_i + \epsilon \cdot \max(0, u_i(a_i, \sigma_{-i}) - u_i(\sigma))}{\text{normalisation}}$$

2. **Montrer que $f$ est continue** : Utilise la continuite des gains esperes

3. **Appliquer Brouwer** : $f$ a un point fixe $\sigma^*$

4. **Verifier que $\sigma^*$ est Nash** : Au point fixe, les perturbations sont nulles, donc $\sigma^*$ est meilleure reponse

### 5.2 Formalisation du theoreme d'existence

In [ ]:
run_lean("""
-- Structure de la preuve d'existence de Nash

-- 1. Definition d'un jeu fini
structure FiniteGame where
  numPlayers : Nat
  numActions : Fin numPlayers → Nat
  payoff : (i : Fin numPlayers) → ((j : Fin numPlayers) → Fin (numActions j)) → Int

-- 2. Profil de strategies mixtes (produit de simplexes)
def MixedProfile (g : FiniteGame) := 
  (i : Fin g.numPlayers) → Fin (g.numActions i) → Float

-- 3. Fonction de meilleure reponse perturbee (simplifiee)
-- En realite, cette fonction est plus complexe et necessite
-- de nombreux lemmes sur la continuite
def perturbedBR (g : FiniteGame) (σ : MixedProfile g) : MixedProfile g :=
  σ  -- Placeholder - la vraie fonction est dans math-xmum/Brouwer

-- 4. Theoreme d'existence (enonce)
-- La preuve complete est dans math-xmum/Brouwer/Nash.lean
theorem nash_equilibrium_exists (g : FiniteGame) 
    (h_pos : ∀ i, g.numActions i > 0) :
    ∃ σ : MixedProfile g, True := by  -- Simplifie
  -- Etapes :
  -- 1. Construire f : perturbedBR
  -- 2. Montrer f continue
  -- 3. Appliquer Brouwer sur produit de simplexes
  -- 4. Verifier que le point fixe est Nash
  exact ⟨perturbedBR g (fun _ _ => 0.5), trivial⟩

#check nash_equilibrium_exists
""")

### 5.3 Autres formalisations

| Depot | Contenu | Lean |
|-------|---------|------|
| [math-xmum/Brouwer](https://github.com/math-xmum/Brouwer) | Nash complet via Scarf | Lean 4 |
| [harfe/fixed-point-theorems-lean4](https://github.com/harfe/fixed-point-theorems-lean4) | Brouwer + Kakutani | Lean 4 |
| [MixedMatched/formalizing-game-theory](https://github.com/MixedMatched/formalizing-game-theory) | Definitions N-joueurs | Lean 4 |

<a id="6-exercices"></a>
## 6. Exercices

### Exercice 1 : Verifier la convexite du simplexe

Prouver que si $x, y \in \Delta^{n-1}$ et $\lambda \in [0,1]$, alors $\lambda x + (1-\lambda) y \in \Delta^{n-1}$.

### Exercice 2 : Point fixe dans un jeu 2x2

Pour le jeu Matching Pennies, construire explicitement la fonction de meilleure reponse perturbee et verifier que $(0.5, 0.5)$ est proche d'un point fixe.

### Exercice 3 : Contre-exemple a Brouwer

Expliquer pourquoi le theoreme de Brouwer ne s'applique pas si :
- Le domaine n'est pas compact (ex: $\mathbb{R}^n$)
- La fonction n'est pas continue

<a id="7-solutions"></a>
## 7. Solutions

### Solution Exercice 1

In [ ]:
run_lean("""
-- Solution Exercice 1 : Convexite du simplexe

-- Le simplexe est convexe : si x, y sont dans le simplexe,
-- alors λx + (1-λ)y est aussi dans le simplexe pour λ ∈ [0,1]

def inSimplex (n : Nat) (v : Fin n → Float) : Prop :=
  (∀ i, v i >= 0) ∧ (List.finRange n).foldl (fun acc i => acc + v i) 0 = 1

-- Combinaison convexe
def convexCombination (n : Nat) (x y : Fin n → Float) (λ : Float) : Fin n → Float :=
  fun i => λ * x i + (1 - λ) * y i

-- Theoreme (enonce) : le simplexe est convexe
-- La preuve complete necessite des lemmes sur Float
theorem simplex_convex (n : Nat) (x y : Fin n → Float) (λ : Float)
    (hx : inSimplex n x) (hy : inSimplex n y)
    (hλ : 0 <= λ ∧ λ <= 1) :
    inSimplex n (convexCombination n x y λ) := by
  constructor
  · -- Non-negativite : λ*x_i + (1-λ)*y_i >= 0
    intro i
    -- Comme x_i >= 0, y_i >= 0, λ >= 0, 1-λ >= 0
    -- leur combinaison est >= 0
    sorry  -- Necessite arithmetique sur Float
  · -- Somme = 1 : sum(λ*x + (1-λ)*y) = λ*sum(x) + (1-λ)*sum(y) = λ*1 + (1-λ)*1 = 1
    sorry

#check @simplex_convex
""")

### Solution Exercice 2

In [ ]:
# Solution Exercice 2 : Point fixe dans Matching Pennies
import numpy as np

# Matrice de gains du joueur 1 (Matching Pennies)
# J1 veut matcher, J2 veut mismatcher
U1 = np.array([[1, -1], [-1, 1]])  # J1
U2 = np.array([[-1, 1], [1, -1]])  # J2

def expected_payoff(sigma1, sigma2, U):
    """Calcule le gain espere."""
    return sigma1 @ U @ sigma2

def perturbed_br(sigma1, sigma2, U, epsilon=0.1):
    """Calcule la meilleure reponse perturbee."""
    # Gain espere actuel
    current = expected_payoff(sigma1, sigma2, U)
    
    # Perturbation vers les actions avec gain superieur
    perturbed = sigma1.copy()
    for a in range(len(sigma1)):
        pure_a = np.zeros(len(sigma1))
        pure_a[a] = 1
        gain_a = expected_payoff(pure_a, sigma2, U)
        if gain_a > current:
            perturbed[a] += epsilon * (gain_a - current)
    
    # Normaliser pour rester dans le simplexe
    return perturbed / perturbed.sum()

# Point de depart
sigma1 = np.array([0.5, 0.5])
sigma2 = np.array([0.5, 0.5])

# Verifier que (0.5, 0.5) est un point fixe
br1 = perturbed_br(sigma1, sigma2, U1)
br2 = perturbed_br(sigma2, sigma1, U2.T)

print(f"sigma1 = {sigma1}")
print(f"BR(sigma1) = {br1}")
print(f"sigma2 = {sigma2}")
print(f"BR(sigma2) = {br2}")
print(f"\n(0.5, 0.5) est un point fixe : {np.allclose(sigma1, br1) and np.allclose(sigma2, br2)}")

### Solution Exercice 3

In [ ]:
# Solution Exercice 3 : Contre-exemples a Brouwer

print("=" * 60)
print("CONTRE-EXEMPLES AU THEOREME DE BROUWER")
print("=" * 60)

print("""
1. DOMAINE NON COMPACT (R au lieu de [0,1])
   
   f(x) = x + 1  sur R
   
   Cette fonction est continue mais n'a pas de point fixe :
   f(x) = x  =>  x + 1 = x  =>  1 = 0  (contradiction)
   
   Le probleme : R n'est pas borne, la fonction peut "fuir a l'infini".

2. FONCTION NON CONTINUE
   
   f(x) = 1 - x  si x < 0.5
         = 1 - x  si x >= 0.5
   
   Ah non, celle-ci est continue. Prenons plutot :
   
   f(x) = 0  si x < 0.5
        = 1  si x >= 0.5
   
   sur [0, 1]. Discontinue en x = 0.5.
   
   Point fixe : f(x) = x
   - Si x < 0.5 : 0 = x => x = 0. Verifions : f(0) = 0. OK, mais...
   - Attendez, x = 0 EST un point fixe !
   
   Meilleur contre-exemple :
   f(x) = 0.75  si x <= 0.5
        = 0.25  si x > 0.5
   
   Point fixe ? f(x) = x
   - x <= 0.5 : 0.75 = x => x = 0.75, mais 0.75 > 0.5, contradiction
   - x > 0.5 : 0.25 = x => x = 0.25, mais 0.25 <= 0.5, contradiction
   
   Pas de point fixe car f est discontinue.
""")

<a id="8-resume"></a>
## 8. Resume

### Theoremes cles

| Theoreme | Enonce | Application |
|----------|--------|-------------|
| **Brouwer** | $f: K \to K$ continue sur compact convexe $\Rightarrow$ point fixe | Base de la preuve |
| **Kakutani** | Extension aux correspondances | Generalisation |
| **Nash** | Tout jeu fini a un equilibre mixte | Consequence directe |

### Structure de la preuve

```
1. Construire f : produit de simplexes → produit de simplexes
2. Montrer que f est continue
3. Appliquer Brouwer : f a un point fixe σ*
4. Verifier : σ* est un equilibre de Nash
```

### Ressources

- [math-xmum/Brouwer](https://github.com/math-xmum/Brouwer) - Formalisation complete en Lean 4
- [harfe/fixed-point-theorems-lean4](https://github.com/harfe/fixed-point-theorems-lean4) - Brouwer + Kakutani
- Nash, J. (1950). "Equilibrium Points in N-Person Games"

### Prochaines etapes

Dans le notebook suivant (**GameTheory-18-Lean-CombinatorialGames**), nous explorerons :
- Les jeux combinatoires (PGame dans mathlib)
- Le theoreme de Sprague-Grundy
- Le jeu de Nim et sa formalisation

---

**Navigation** : [← GameTheory-16-Lean-Definitions](GameTheory-16-Lean-Definitions.ipynb) | [Index](GameTheory-1-Setup.ipynb) | [GameTheory-18-Lean-CombinatorialGames →](GameTheory-18-Lean-CombinatorialGames.ipynb)